In [100]:
import os
import sys
project_path = os.path.abspath(os.path.join('../..'))
if project_path not in sys.path:
    sys.path.append(project_path)
# os.path.split(os.path.split(os.getcwd())[0])[0]

In [101]:
import src.CONSTANTS as CNSTS
import os
import src.CONF as CONF
import json
import math
from src.stk import zillion
from src.utils import email_info
from datetime import datetime
from dotenv import load_dotenv
from binance.client import Client
import csv
import talib

In [102]:
# 1. Following constant product formula for market making, x * y = k
# 2. where, x is the amount of asset 1, and y is the amount of asset 2 pooled in.
# 3. Assuming price of asset 1 in relation to asset 2 is m, 
# 4. then y = m * x, therefore k = x * m * x 
# 5. Considering a change in relative price of asset 1 to asset 2 i.e. m -> m',
# 6. Updated state of pooled assets x -> x', y-> y'
# 7. x' * m' * x' = k, therefore x' * m' * x' = x * m * x
# 8. x' = sqr_rt((x * m * x)/m')
# 9. Assuming m' = n * m, where it can be read as new relative price of asset 1 is n% of the current price
# 10. x' = sqr_rt((x * x)/n) 
# 11. y' = k/x'
# 12. Lets consider the $ value of asset 2 unit as v, asset 1 unit $ value (u) would be v * m
# 13. Therefore, total worth of the assets would be x * (v * m) + y * v  
# 14. Considering the updated $ value of asset 2 as u'
# 15. Difference in assets worth would be: (x' * (u' * m') + y' * u') - (x * (u * m) + y * u) 
# 16. This difference is referred to as impermanent loss i.e. comparing the buy and hold strategy vs putting assets in the pool

In [103]:
# Experiment with different price curves, 
# e.g. assuming a delta of 1-5% change in reaching the target price change in the assets

In [104]:
def get_updated_price(price, price_delta):
    return price * (1 + price_delta)

def k(x, y):
    return x * y

def worth(x, u, y, v): 
    return x * u + y * v

def worth_with_m(x, y, v, m): 
    return x * (v * m) + y * v

def get_updated_x_y(x, y, m_dash):
    m = y/x
    product = k(x,y)
    x_dash = math.sqrt((m/m_dash) * x * x)
    y_dash = product/x_dash
    return x_dash, y_dash

def impermanent_loss(x, y, x_dash, y_dash, u_dash, v_dash):
    original_worth = worth(x, u_dash, y, v_dash)
    updated_worth = worth(x_dash, u_dash, y_dash, v_dash)
    return (updated_worth - original_worth), original_worth

In [105]:
a1_pool_weight = 0.5
a2_pool_weight = 0.5

a1_borrow_rate = 5 # Rate of borrowing an asset from a centralized exchange or a smart contract etc.
a2_borrow_rate = 10

# asset1_constraint_range = 5 # Percentage change in liquidity +- both, can be separated as well


# Assuming following, user pool in: 10K $, zil at 0.17$, Zil-USD pool
# change of 50% price in Zil
# Goal is to maximize time stayed in the pool, while minimizing the impermanent loss

# Assume asset 1 as x, and asset 2 as y following the above formulation
u = 0.17 # $ price of asset 1
v = 1 # $ price of asset 2
m = u/v
delta_u = 0.5 # Percent change
u_dash = get_updated_price(u, delta_u)
delta_v = 0.0 # Percent change
v_dash = get_updated_price(v, delta_v)                           
m_dash = u_dash/v_dash

In [106]:
user_amount_in_usd = 10000
user_x = (user_amount_in_usd * a1_pool_weight) / u
user_y = (user_amount_in_usd * a2_pool_weight) / v
print("user x: ", user_x, " user y: ", user_y)
user_worth = worth(user_x, u, user_y, v)
print("Initial user worth", user_worth)

user x:  29411.76470588235  user y:  5000.0
Initial user worth 10000.0


In [107]:
# To ignore slippage for the simulation, assume a very high amount of liquidity in the select pool
# Assuming a high asset1-asset2 dex pool, i.e. <0.1% slippage for any invested amount
pool_approx_liq_multiplier = 1000
pool_x = user_x * pool_approx_liq_multiplier
pool_y = user_y * pool_approx_liq_multiplier
print("pool x: ", pool_x, " pool y: ", pool_y)

pool x:  29411764.705882348  pool y:  5000000.0


In [108]:
pool_x_dash, pool_y_dash = get_updated_x_y(pool_x, pool_y, m_dash)
print("pool x: ", pool_x_dash, " pool y: ", pool_y_dash)

pool x:  24014605.32140371  pool y:  6123724.356957944


In [109]:
user_x_dash = pool_x_dash / pool_approx_liq_multiplier
user_y_dash = pool_y_dash / pool_approx_liq_multiplier
print("user x: ", user_x_dash, " user y: ", user_y_dash)
user_updated_worth = worth(user_x_dash, u_dash, user_y_dash, v_dash) 
print("Updated user worth", user_updated_worth)     

user x:  24014.60532140371  user y:  6123.724356957944
Updated user worth 12247.44871391589


In [110]:
print("Asset 1 price with ", round(delta_u*100, 2), " % change:" , u, " -> ", u_dash)
print("Asset 2 price with ", round(delta_v*100, 2), " % change:" , v, " -> ", v_dash)

Asset 1 price with  50.0  % change: 0.17  ->  0.255
Asset 2 price with  0.0  % change: 1  ->  1.0


In [111]:
user_buy_hold_worth = worth(user_x, u_dash, user_y, v_dash)
il = (user_updated_worth - user_buy_hold_worth)/user_buy_hold_worth
print("Impermanent loss %: ", round(il, 2))
print("User buy and hold worth: ", round(user_buy_hold_worth, 2))

Impermanent loss %:  -0.02
User buy and hold worth:  12500.0


In [112]:
print("Asset 1 % change: ", round(((user_x_dash - user_x)/user_x)*100, 2))
print("Asset 2 % change: ", round(((user_y_dash - user_y)/user_y)*100, 2))

Asset 1 % change:  -18.35
Asset 2 % change:  22.47


In [54]:
def get_binance_data():
    load_dotenv()
    personal_env = os.getenv(CONF.PERSONAL_ENV_FILE)
    load_dotenv(personal_env)

    api_key = os.getenv(CONF.BIN_WALLET_1_API_1["API_KEY"])
    api_secret = os.getenv(CONF.BIN_WALLET_1_API_1["API_SECRET"])

    client = Client(api_key, api_secret)

    start_date = "5 Feb, 2021"
    end_date = "20 Feb, 2021"
    asset_pair = 'ZILUSDT'
    data_type = "kline_15m"

    # candles = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_30MINUTE)
    candles = client.get_historical_klines(asset_pair,
                                           Client.KLINE_INTERVAL_15MINUTE,
                                           start_date,
                                           end_date
                                           )
    return candles
    

In [ ]:
# product = pool_token_count * pool_zil_count
# pool_token_amount_after_tx = pool_token_count + token_amount_to_be_sold
# token_price_for_tx = (pool_zil_count - product / pool_token_amount_after_tx) / token_amount_to_be_sold
# token_price_for_tx = token_price_for_tx - ((fees / 100) * token_price_for_tx)

In [ ]:
# what 1 transaction would bring a change of price of x% in asset1 in the pool
# a buy order of asset1 would increase the price, while a sell order decreases the price
# calculate the amount of asset1 tokens need to be bought from this pool to 
# increase asset1 price by x%
# Assuming a high asset1-asset2 dex pool, i.e. <1% slippage for any invested amount
# roi through liquidity mining is defined by the time spent by the liquidity spent in the pool, which in turn
# gives the amount of fees earned and the governance token rewards
# track binance price continuously, if it changes by 2%, then remove the liquidity
# also note that there could be a lag between the binance prices and the dex price or vice versa (in some cases)
# leading to arbitrage opportunities